Function to run faster on dataset

In [14]:
import torch
import torchvision
import fiftyone as fo
from PIL import Image
from torchvision.transforms import functional as func


def add_detect_to_dataset(dataset, num_samples = None, map_labels={}, predictions_label = "predictions"):
    if(num_samples is None):
        num_samples = len(dataset)

    # Run the model on GPU if it is available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Load a pre-trained Faster R-CNN model
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    model.to(device)
    model.eval()

    print("Model ready")

    # Choose a random subset of 100 samples to add predictions to
    predictions_view = dataset.take(num_samples, seed=51)

    # Get class list
    classes = dataset.distinct("predictions.detections.label") #dataset.default_classes
    print(classes)

    # Add predictions to samples
    with fo.ProgressBar() as pb:
        for sample in pb(predictions_view):
            # Load image
            image = Image.open(sample.filepath)
            image = func.to_tensor(image).to(device)
            c, h, w = image.shape

            # Perform inference
            preds = model([image])[0]
            labels = preds["labels"].cpu().detach().numpy()
            scores = preds["scores"].cpu().detach().numpy()
            boxes = preds["boxes"].cpu().detach().numpy()

            # Convert detections to FiftyOne format
            detections = []

            keys = map_labels.keys()
            true_labels = []
            true_scores = []
            true_boxes = []
            if(len(map_labels) == 0):
                true_labels = labels
                true_scores = scores
                true_boxes = boxes
            else:
                for label, score, box in zip(labels, scores, boxes):
                    if(label in keys):
                        true_labels.append(map_labels[label])
                        true_scores.append(score)
                        true_boxes.append(box)

            for label, score, box in zip(true_labels, true_scores, true_boxes):
                # Convert to [top-left-x, top-left-y, width, height]
                # in relative coordinates in [0, 1] x [0, 1]
                x1, y1, x2, y2 = box
                rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]

                detections.append(
                    fo.Detection(
                        label=classes[label],
                        bounding_box=rel_box,
                        confidence=score
                    )
                )

            # Save predictions to dataset
            sample[predictions_label] = fo.Detections(detections=detections)
            sample.save()

        return dataset


Function to read coco from file

In [15]:
import fiftyone as fo
import fiftyone.zoo as foz
dataset = foz.load_zoo_dataset('quickstart')

Dataset already downloaded
Loading existing dataset 'quickstart'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [16]:
dataset = add_detect_to_dataset(dataset,num_samples=None, map_labels={},predictions_label="predictions_faster")

Model ready
['airplane', 'apple', 'backpack', 'banana', 'baseball glove', 'bear', 'bed', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl', 'broccoli', 'bus', 'cake', 'car', 'carrot', 'cat', 'cell phone', 'chair', 'clock', 'couch', 'cow', 'cup', 'dining table', 'dog', 'donut', 'elephant', 'fire hydrant', 'fork', 'frisbee', 'giraffe', 'hair drier', 'handbag', 'horse', 'hot dog', 'keyboard', 'kite', 'knife', 'laptop', 'microwave', 'motorcycle', 'mouse', 'orange', 'oven', 'parking meter', 'person', 'pizza', 'potted plant', 'refrigerator', 'remote', 'sandwich', 'scissors', 'sheep', 'sink', 'skateboard', 'skis', 'snowboard', 'spoon', 'sports ball', 'stop sign', 'suitcase', 'surfboard', 'teddy bear', 'tennis racket', 'tie', 'toaster', 'toilet', 'toothbrush', 'traffic light', 'train', 'truck', 'tv', 'umbrella', 'vase', 'wine glass', 'zebra']
   1% |\----------------|   2/200 [4.9s elapsed, 8.1m remaining, 0.6 samples/s] 


IndexError: list index out of range

In [13]:
print(dataset)

Name:        quickstart
Media type:  image
Num samples: 200
Persistent:  False
Tags:        ['small']
Sample fields:
    id:                fiftyone.core.fields.ObjectIdField
    filepath:          fiftyone.core.fields.StringField
    tags:              fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    uniqueness:        fiftyone.core.fields.FloatField
    predictions:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    my_uniqueness:     fiftyone.core.fields.FloatField
    eval_tp:           fiftyone.core.fields.IntField
    eval_fp:           fiftyone.core.fields.IntField
    eval_fn:           fiftyone.core.fields.IntField
    mistakenness:      fiftyone.core.fields.FloatField
    possible_missing:  fiftyone.core.fields.IntField
  